# Digit Recognizer using vanilla RNN

In [1]:
import tensorflow as tf
import numpy as np 
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


#### some constants

In [3]:
element_size = 28
time_steps = 28
num_classes = 10
batch_size = 128
hidden_layer_size = 128

In [4]:
#log directory 
LOG_DIR = "logs/RNN_with_summaries"

In [5]:
_inputs = tf.placeholder(tf.float32, shape=[None, time_steps, element_size], name='inputs')
y = tf.placeholder(tf.float32, shape=[None, num_classes], name="labels")

In [6]:
# tensorboard 
def variable_summaries(var):
    with tf.name_scope("summaries"):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope("stddev"):
            stddev= tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar("stddev", stddev)
        tf.summary.scalar("max", tf.reduce_max(var))
        tf.summary.scalar("min", tf.reduce_min(var))
        tf.summary.histogram("histogram", var)

In [7]:
with tf.name_scope("rnn_weights"):
    with tf.name_scope("W_x"):
        Wx = tf.Variable(tf.zeros([element_size, hidden_layer_size]))
        variable_summaries(Wx)
    
    with tf.name_scope("W_h"):
        Wh = tf.Variable(tf.zeros([hidden_layer_size, hidden_layer_size]))
        variable_summaries(Wh)
    
    with tf.name_scope("bias"):
        bias = tf.Variable(tf.zeros([hidden_layer_size]))
        variable_summaries(bias)

In [8]:
def rnn_step(previous_hidden_state, x):
    current_hidden_state = tf.tanh(tf.matmul(previous_hidden_state, Wh) + 
                                  tf.matmul(x, Wx) + bias)
    return current_hidden_state

In [9]:
processed_input = tf.transpose(_inputs, perm=[1, 0, 2])

initial_hidden = tf.zeros([batch_size, hidden_layer_size])

all_hidden_states = tf.scan(rnn_step, processed_input, initializer=initial_hidden,
                           name="states")

In [10]:
# weights for output layer
with tf.name_scope("linear_layer_weights") as scope:
    with tf.name_scope("W_linear"):
        Wl = tf.Variable(tf.truncated_normal([hidden_layer_size, num_classes], mean=0, stddev=0.01))
        variable_summaries(Wl)
    
    with tf.name_scope("bias_linear"):
        bias_linear = tf.Variable(tf.truncated_normal([num_classes], mean=0, stddev=0.01))
        variable_summaries(bias_linear)

In [11]:
def get_linear_layer(hidden_state):
    return tf.matmul(hidden_state, Wl) + bias_linear

In [12]:
with tf.name_scope("linear_layer_weights") as scope:
    all_outputs = tf.map_fn(get_linear_layer, all_hidden_states)
    output = all_outputs[-1]
    tf.summary.histogram("outputs", output)

In [13]:
with tf.name_scope("cross_entropy"):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=y))
    tf.summary.scalar("cross_entropy", cross_entropy)

with tf.name_scope("train_step"):
    train_step = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cross_entropy)
    
with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(y, 1))
    accuracy = (tf.reduce_mean(tf.cast(correct_prediction, tf.float32)))*100
    tf.summary.scalar("accuracy", accuracy)
    
merged = tf.summary.merge_all()

In [14]:
# running the vanilla model 
test_data = mnist.test.images[:batch_size].reshape((-1, time_steps, element_size))
test_labels = mnist.test.labels[:batch_size]

with tf.Session() as sess:
    train_writer = tf.summary.FileWriter(LOG_DIR + "/train", graph=tf.get_default_graph())
    test_writer = tf.summary.FileWriter(LOG_DIR + "/test", graph=tf.get_default_graph())
    
    sess.run(tf.global_variables_initializer())
    
    for i in range(10000):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        
        batch_x = batch_x.reshape((batch_size, time_steps, element_size))
        
        summary, _ = sess.run([merged, train_step], feed_dict={
            _inputs: batch_x, y: batch_y
        })
        train_writer.add_summary(summary, i)
        
        if i%1000 == 0:
            acc, loss = sess.run([accuracy, cross_entropy], feed_dict={
            _inputs: batch_x, y: batch_y
            })
            print "Iteration: " + str(i) +", Minibatch loss: {:.6f}".format(loss)\
            + ", Training Accuracy: {:.5f}".format(acc)
        
        if i%10 == 0:
            summary, acc = sess.run([merged, accuracy], feed_dict={
                _inputs: test_data, y: test_labels
            })
            test_writer.add_summary(summary, i)
            
    test_accuracy = sess.run(accuracy, feed_dict={
                _inputs: test_data, y: test_labels
    })
    print "Test Accuracy:  %i" %test_accuracy

Iteration: 0, Minibatch loss: 2.302722, Training Accuracy: 8.59375
Iteration: 1000, Minibatch loss: 1.267731, Training Accuracy: 46.87500
Iteration: 2000, Minibatch loss: 0.467752, Training Accuracy: 85.93750
Iteration: 3000, Minibatch loss: 0.327170, Training Accuracy: 87.50000
Iteration: 4000, Minibatch loss: 0.300982, Training Accuracy: 91.40625
Iteration: 5000, Minibatch loss: 0.089024, Training Accuracy: 96.09375
Iteration: 6000, Minibatch loss: 0.041998, Training Accuracy: 99.21875
Iteration: 7000, Minibatch loss: 0.050911, Training Accuracy: 99.21875
Iteration: 8000, Minibatch loss: 0.053395, Training Accuracy: 98.43750
Iteration: 9000, Minibatch loss: 0.077494, Training Accuracy: 98.43750
Test Accuracy:  99
